# Target: Compare runs, choose a model, and deploy it to a REST API

- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLflow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform


# Import Required Libraries & Define Optimization Tools

In [ ]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature
#STATUS_OK: A constant that indicates a trial (iteration of the optimization process) was successful
# Trials: A container that stores details of all optimization trials, including:
#Hyperparameters tested
#Corresponding losses
#Best-performing trial so far
# fmin: A function for finding the best hyperparameters
# hp: A class for specifying hyperparameters

# Load and Explore the Dataset

In [17]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


# Split the Data into Training, Validation, and Test Sets
- 80% of the data is used for training, while 20% is reserved for testing.

- The training set is further split into training (80%) and validation (20%).



In [18]:
## Split the data into training,validation and test sets
train,test=train_test_split(data,test_size=0.2,random_state=42)
print(train.shape)
print(test.shape)


(3918, 12)
(980, 12)


In [19]:
## gives multidim array
train[['quality']].values

array([[6],
       [5],
       [6],
       ...,
       [6],
       [6],
       [8]])

In [20]:
train_x=train.drop(['quality'],axis=1).values
##convert a Pandas Series or DataFrame (y_train) into a 1-dimensional NumPy array.
train_y=train[['quality']].values.ravel()

## TEST DATA
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

# Splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.2,random_state=42)

signature=infer_signature(train_x,train_y)


In [21]:
train_x.shape

(3134, 11)

## There are 3 subsections of this section:

# Define an ANN Model

- 64 hidden neurons with ReLU activation allow the model to learn non-linear relationships.
- Feature normalization ensures that all input features have similar scales.

# Compile and Train the Model

- Tracks results using MLflow.
- Logs RMSE score for evaluation

# Log Results in MLflow
- Hyperparameters
- RMSE loss
- Trained model to MLflow


In [ ]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )
    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]



        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

# Run Hyperparameter Optimization
- Uses Hyperopt to find the best combination of hyperparameters.

In [ ]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

# Hyperparameter Search & Best Model Selection with MLflow
- Run Hyperparameter Optimization using Hyperopt (fmin function).
- Track all experiment runs in MLflow and compare their results.
- Identify the best hyperparameter combination and log the model.

In [25]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/02/06 21:22:29 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 24s 511ms/step - loss: 30.4571 - root_mean_squared_error: 5.5188
34/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.8884 - root_mean_squared_error: 3.0101    
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 7.9783 - root_mean_squared_error: 2.6800 - val_loss: 1.6271 - val_root_mean_squared_error: 1.2756

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 1.4264 - root_mean_squared_error: 1.1943
23/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2915 - root_mean_squared_error: 1.1357 
43/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2328 - root_mean_squared_error: 1.1096
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2183 - root_mean_squared_error: 1.1030 - val_loss: 1.1122 - val_root_mean_squared_error: 1.0546

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.8101 - root_mean_squared_error: 0.9001
33/4

# Validate the model before deployment by MlFlow code
- Ensures the model input format is correct before deployment.

In [ ]:
## For inferencing there are 2 ways.
## The first is to use the model directly from MLFLOW website

from mlflow.models import validate_serving_input, convert_input_example_to_serving_input

# Specify the URI of the logged model
model_uri = 'runs:/90d616fec64d45dd949030edf9499fef/model'

# The logged model does not contain an input example.
# Manually generate a serving payload to verify your model prior to deployment.
# Define INPUT_EXAMPLE via assignment with your own input example for the model.
# A valid input example is a data instance suitable for pyfunc prediction.


# Convert the input example to a serving-compatible payload
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate that the serving payload works with the model
validate_serving_input(model_uri, serving_payload)

#print("Serving input validation successful!")


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[6.125681 ],
       [6.3469334],
       [6.356963 ],
       [5.6380205],
       [6.1434917],
       [6.4192905],
       [5.1692076],
       [5.2994676],
       [5.837593 ],
       [4.7733555],
       [6.557067 ],
       [4.447655 ],
       [6.493622 ],
       [5.367612 ],
       [6.26243  ],
       [5.3934517],
       [6.6831446],
       [6.059698 ],
       [5.938624 ],
       [5.161605 ],
       [5.1310406],
       [6.0404353],
       [5.0687013],
       [5.880807 ],
       [5.720621 ],
       [5.4352965],
       [5.0174913],
       [6.0549555],
       [5.5869265],
       [5.258593 ],
       [5.2865286],
       [5.2465878],
       [5.425581 ],
       [5.28242  ],
       [5.465637 ],
       [6.3743725],
       [6.4896107],
       [4.963009 ],
       [5.1539764],
       [5.949724 ],
       [5.3200493],
       [4.9485693],
       [5.784628 ],
       [5.5428934],
       [4.840986 ],
       [5.669524 ],
       [4.991514 ],
       [4.983362 ],
       [5.3769627],
       [4.8520885],


# Load Model and Make Predictions using pyfunc
- Loads the best model and runs predictions on unseen data.

In [29]:
## 2nd one is to load the model.

model=mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame
import pandas as pd
model.predict(pd.DataFrame(test_x))


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[6.125681 ],
       [6.3469334],
       [6.356963 ],
       [5.6380205],
       [6.1434917],
       [6.4192905],
       [5.1692076],
       [5.2994676],
       [5.837593 ],
       [4.7733555],
       [6.557067 ],
       [4.447655 ],
       [6.493622 ],
       [5.367612 ],
       [6.26243  ],
       [5.3934517],
       [6.6831446],
       [6.059698 ],
       [5.938624 ],
       [5.161605 ],
       [5.1310406],
       [6.0404353],
       [5.0687013],
       [5.880807 ],
       [5.720621 ],
       [5.4352965],
       [5.0174913],
       [6.0549555],
       [5.5869265],
       [5.258593 ],
       [5.2865286],
       [5.2465878],
       [5.425581 ],
       [5.28242  ],
       [5.465637 ],
       [6.3743725],
       [6.4896107],
       [4.963009 ],
       [5.1539764],
       [5.949724 ],
       [5.3200493],
       [4.9485693],
       [5.784628 ],
       [5.5428934],
       [4.840986 ],
       [5.669524 ],
       [4.991514 ],
       [4.983362 ],
       [5.3769627],
       [4.8520885],


# Register and Deploy the Best Model using code
- Registers the best model in MLflow’s Model Registry.

In [30]:
# Register the model by code

model_uri = 'runs:/90d616fec64d45dd949030edf9499fef/model'
mlflow.register_model(model_uri=model_uri, name="wine-quality-model")


Successfully registered model 'wine-quality-model'.
Created version '1' of model 'wine-quality-model'.


<ModelVersion: aliases=[], creation_timestamp=1738907995197, current_stage='None', description=None, last_updated_timestamp=1738907995197, name='wine-quality-model', run_id='90d616fec64d45dd949030edf9499fef', run_link=None, source='file:///c:/Users/Gaurob/Desktop/DLMLFLOW/mlruns/997668502739364301/90d616fec64d45dd949030edf9499fef/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>